In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [2]:
timesteps = 14
startDay = 350

In [3]:
calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
sell_prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')

sales_train_evaluation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')

In [4]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [5]:
sales_train_validation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')

In [6]:
dt = downcast_dtypes(sales_train_evaluation)

In [7]:
dt = dt.T    
dt.head(8)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
item_id,HOBBIES_1_001,HOBBIES_1_002,HOBBIES_1_003,HOBBIES_1_004,HOBBIES_1_005,HOBBIES_1_006,HOBBIES_1_007,HOBBIES_1_008,HOBBIES_1_009,HOBBIES_1_010,...,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
dept_id,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,...,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3
cat_id,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,...,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS
store_id,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,...,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3
state_id,CA,CA,CA,CA,CA,CA,CA,CA,CA,CA,...,WI,WI,WI,WI,WI,WI,WI,WI,WI,WI
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0


In [8]:
dt = dt[6 + startDay:]
dt.head(5)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,...,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,7,1,0,1,0,1,1,0,0


In [9]:
daysBeforeEvent = pd.DataFrame(np.zeros((1969,1)))

In [10]:
for x,y in calendar.iterrows():
    if((pd.isnull(calendar["event_name_1"][x])) == False):
           daysBeforeEvent[0][x-1] = 1 

In [11]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [12]:
snap_CA=pd.DataFrame(np.zeros((1969,1)))
snap_TX=pd.DataFrame(np.zeros((1969,1)))
snap_WI=pd.DataFrame(np.zeros((1969,1)))

In [13]:
for x,y in calendar.iterrows():
    if(((calendar["snap_CA"][x])) == 1):
           snap_CA[0][x-1] = 1 

In [14]:
for x,y in calendar.iterrows():
    if(((calendar["snap_WI"][x])) == 1):
           snap_WI[0][x-1] = 1 

In [15]:
for x,y in calendar.iterrows():
    if(((calendar["snap_TX"][x])) == 1):
           snap_TX[0][x-1] = 1 

In [16]:
#"calendar" won't be used anymore. 
del calendar

In [17]:
snap_CA_test = snap_CA[1941:1969]
#"daysBeforeEvent" will be used for training as a feature.
snap_CA = snap_CA[startDay:1941]

In [18]:
snap_WI_test = snap_WI[1941:1969]
#"daysBeforeEvent" will be used for training as a feature.
snap_WI = snap_WI[startDay:1941]

In [19]:
snap_TX_test = snap_TX[1941:1969]
#"daysBeforeEvent" will be used for training as a feature.
snap_TX = snap_TX[startDay:1941]

In [20]:
daysBeforeEventTest = daysBeforeEvent[1941:1969]
#"daysBeforeEvent" will be used for training as a feature.
daysBeforeEvent = daysBeforeEvent[startDay:1941]

In [21]:
daysBeforeEvent.columns = ["oneDayBeforeEvent"]
daysBeforeEvent.index = dt.index

In [22]:
snap_CA.columns = ["snap_CA"]
snap_CA.index = dt.index

In [23]:
snap_WI.columns = ["snap_WI"]
snap_WI.index = dt.index

In [24]:
snap_TX.columns = ["snap_TX"]
snap_TX.index = dt.index

In [25]:
dt = pd.concat([dt, daysBeforeEvent,snap_CA,snap_WI,snap_TX], axis = 1)


dt.columns

Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,
       ...
                     30484,               30485,               30486,
                     30487,               30488,               30489,
       'oneDayBeforeEvent',           'snap_CA',           'snap_WI',
                 'snap_TX'],
      dtype='object', length=30494)

In [26]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
dt_scaled = sc.fit_transform(dt)

In [27]:
X_train = []
y_train = []
for i in range(timesteps, 1941 - startDay):
    X_train.append(dt_scaled[i-timesteps:i])
    y_train.append(dt_scaled[i][0:30490])

In [28]:
del dt_scaled

In [29]:
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1577, 14, 30494)
(1577, 30490)


In [30]:
import tensorflow as tf

2021-10-09 05:28:57.136779: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-09 05:28:57.136942: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout


In [32]:
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
layer_1_units=50
regressor.add(LSTM(units = layer_1_units, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
layer_3_units=400
regressor.add(LSTM(units = layer_3_units, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
layer_3_units=400
regressor.add(LSTM(units = layer_3_units))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 30490))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
epoch_no=32
batch_size_RNN=44
regressor.fit(X_train, y_train, epochs = epoch_no, batch_size = batch_size_RNN)

2021-10-09 05:29:01.849290: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-09 05:29:01.853304: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-09 05:29:01.853348: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-09 05:29:01.853375: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (61d17ee881bc): /proc/driver/nvidia/version does not exist
2021-10-09 05:29:01.854485: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Epoch 1/32
36/36 [==============================] - 27s 609ms/step - loss: 0.0199
Epoch 2/32
36/36 [==============================] - 21s 581ms/step - loss: 0.0160
Epoch 3/32
36/36 [==============================] - 21s 576ms/step - loss: 0.0153
Epoch 4/32
36/36 [==============================] - 21s 586ms/step - loss: 0.0150
Epoch 5/32
36/36 [==============================] - 21s 584ms/step - loss: 0.0148
Epoch 6/32
36/36 [==============================] - 21s 577ms/step - loss: 0.0146
Epoch 7/32
36/36 [==============================] - 21s 585ms/step - loss: 0.0144
Epoch 8/32
36/36 [==============================] - 21s 577ms/step - loss: 0.0141
Epoch 9/32
36/36 [==============================] - 21s 576ms/step - loss: 0.0141
Epoch 10/32
36/36 [==============================] - 21s 593ms/step - loss: 0.0140
Epoch 11/32
36/36 [==============================] - 21s 580ms/step - loss: 0.0139
Epoch 12/32
36/36 [==============================] - 21s 573ms/step - loss: 0.0139
Epoch 13/32
3

In [33]:
inputs= dt[-timesteps:]
inputs = sc.transform(inputs)

In [34]:
X_test = []
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []
print(X_test.shape)

for j in range(timesteps,timesteps + 28):
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_stock_price = regressor.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30494))
    testInput = np.column_stack((np.array(predicted_stock_price),daysBeforeEventTest[0][1941 + j - timesteps], snap_CA_test[0][1941 + j - timesteps],snap_WI_test[0][1941 + j - timesteps],snap_TX_test[0][1941 + j - timesteps]))
    X_test = np.append(X_test, testInput).reshape(1,j+1 ,30494)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_stock_price)


(1, 14, 30494)


In [35]:
dataPath = "/kaggle/input/m5-forecasting-accuracy/"

In [36]:
import time

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))

submission = submission.T
    
submission = pd.concat((submission, submission), ignore_index=True)

sample_submission = pd.read_csv(dataPath + "/sample_submission.csv")
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

submission.to_csv("submission.csv", index=False)

In [37]:
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.987258,0.907522,0.855718,0.851227,0.941525,1.279286,1.293656,0.950125,0.869013,...,0.918942,1.272523,1.239399,0.935563,0.868849,0.824669,0.814865,0.894559,1.208027,1.189286
1,HOBBIES_1_002_CA_1_validation,0.268116,0.304501,0.293827,0.284824,0.316871,0.432050,0.349945,0.279872,0.320550,...,0.353918,0.467513,0.370625,0.286291,0.327485,0.327756,0.329184,0.366439,0.476086,0.374077
2,HOBBIES_1_003_CA_1_validation,0.555646,0.505726,0.547263,0.653012,0.842296,1.121379,0.971419,0.542643,0.480142,...,0.848755,1.137706,0.953474,0.532046,0.496836,0.552585,0.653566,0.830005,1.089145,0.892362
3,HOBBIES_1_004_CA_1_validation,2.136581,1.490046,1.291162,1.304827,1.497584,2.321727,3.836878,2.164149,1.472658,...,1.534785,2.377651,3.785918,2.032299,1.426315,1.275418,1.341533,1.554465,2.359739,3.723386
4,HOBBIES_1_005_CA_1_validation,1.296787,1.150302,1.120056,1.120180,1.134436,1.196300,1.568214,1.224519,1.084606,...,1.092934,1.187104,1.545159,1.191575,1.076847,1.063580,1.066196,1.080040,1.147137,1.482070


In [38]:
ss

NameError: name 'ss' is not defined